# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [1]:
import sys

sys.stdout.write('Hello, my lady! Hello, my honey!')

Hello, my lady! Hello, my honey!

32

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [2]:
output = open("./some_test_data.txt", 'w')

In [3]:
output.write('123')

3

In [4]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [5]:
import sys
from datetime import datetime  # Получаем класс datetime из модуля datetime

original_write = sys.stdout.write

def my_write(string_text):
    current_datetime = datetime.now().strftime('[%Y-%m-%d %H:%M:%S]: ')
    if string_text != '\n':  # Без этого принт будет выводить дату и время в начале и конце строки
        original_write(current_datetime + string_text)

sys.stdout.write = my_write

In [6]:
print('1, 2, 3')

[2022-11-24 13:59:10]: 1, 2, 3

In [3]:
sys.stdout.write = original_write

Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [7]:
def timed_output(function):
    def wrapper(string_text):
        sys.stdout.write = my_write
        function(string_text)
    return wrapper

In [8]:
@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')

In [9]:
print_greeting('is it me you looking for?')

[2022-11-24 13:59:49]: Hello, is it me you looking for?!

Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [14]:
import sys

def redirect_output(filepath):
    def wrapper(function):
        try:
            sys.stdout = open(filepath, 'w')
            function()
        finally:
            sys.stdout.close()
            sys.stdout = sys.__stdout__  
    return wrapper

In [15]:
@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()